# Use web-scraping to validate if a given TIN number (Ex.07490304055)  is correct or not 

 http://www.tinxsys.com/TinxsysInternetWeb/searchByTin_Inter.jsp

#If correct, write out the available details into a text file and create a new folder in AWS S3 or Dropbox with time stamp as the name of the folder and store each text file using the APIs provided by AWS or Dropbox.

In [429]:
import mechanize
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import urllib2
import pandas as pd
import datetime
import re
import datetime
import dropbox

In [430]:
def get_tin_details(TINNo):
    """
    To get tin details for TIN No and return content
    """
    url = "http://www.tinxsys.com/TinxsysInternetWeb/searchByTin_Inter.jsp"
    br = mechanize.Browser()
    br.set_handle_robots(False) # ignore robots
    br.open(url)
    br.select_form(name="searchByTin")
    br["tinNumber"] = TINNo
    br.addheaders = [('User-agent', 'Firefox')]
    res = br.submit()
    content = res.read()
    soup = BeautifulSoup(content,'lxml')
    soup.prettify()
    table_list = soup.find_all("table")
    table_data_final = None
    content_table = table_list[2] # In web page 3rd table have all content 
    table_data = [j.text.replace('\r\n', '').replace('\n','').replace(u'\xa0', '').strip() for j in content_table.find_all("tr")]
    table_data_final = [re.sub('\s{4,}', ' : ', line) for line in table_data]
    return table_data_final

In [431]:
#read input of tin no list from input.csv file

In [432]:
input_list = pd.read_csv('input.csv')
input_list

,TIN_NO
0,'07490304055'
1,'07490304054'


In [433]:
TIN_no_list = [str(i).replace("'","") for i in input_list.TIN_NO]

In [434]:
TIN_no_list

['07490304055', '07490304054']

# Manual List of TIN NO instead of read from input.csv file

In [435]:
#TIN_no_list = ['07490304055','07490304054','07490304053']

In [441]:
def write_outputin_dropbox(file_name,content):
    """
    Replace access token with your app auth access token
    """
    access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
    client = dropbox.client.DropboxClient(access_token)
    response = client.put_file(file_name,content)
    print response

In [442]:
def write_tin_details():
    """
    Write tin details for all TIN NO in input.csv file 
    """
    current_timestamp = str(datetime.datetime.now())
    for TIN_NO in TIN_no_list:
        try:
            content = get_tin_details(TIN_NO) 
            file_name = "outout_%s_%s.txt" %(TIN_NO,current_timestamp)
            content_text = ",".join(content)
            write_outputin_dropbox(file_name,content_text)
            
            #Write all output in output.txt file with timestamp   
            #pd.DataFrame({'content':content,'timestamp':current_timestamp}).to_csv('output.txt',mode='a',index=False)
    
            #To write output in different different txt file as TIN_NO_TIMESTAMP.txt
            # pd.DataFrame({'content':content,'timestamp':current_timestamp}).to_csv(file_name,header=['content','timestamp'],index=False)
        except:
            print "Not able to get TIN details for TIN NO %s " %(TIN_NO)
    
    print "Operation compleated successfully"      

In [443]:
#Execute main function
write_tin_details()

/root/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: You are using a deprecated client. Please use the new v2 client located at dropbox.Dropbox.


{u'icon': u'page_white_text', u'bytes': 388, u'thumb_exists': False, u'rev': u'31441cc39a', u'modified': u'Sun, 11 Dec 2016 12:52:43 +0000', u'shareable': False, u'client_mtime': u'Sun, 11 Dec 2016 12:52:43 +0000', u'path': u'/outout_07490304055_2016-12-11 18:22:40.590358.txt', u'is_dir': False, u'size': u'388 bytes', u'root': u'dropbox', u'mime_type': u'text/plain', u'revision': 49}
{u'icon': u'page_white_text', u'bytes': 49, u'thumb_exists': False, u'rev': u'32441cc39a', u'modified': u'Sun, 11 Dec 2016 12:52:44 +0000', u'shareable': False, u'client_mtime': u'Sun, 11 Dec 2016 12:52:44 +0000', u'path': u'/outout_07490304054_2016-12-11 18:22:40.590358.txt', u'is_dir': False, u'size': u'49 bytes', u'root': u'dropbox', u'mime_type': u'text/plain', u'revision': 50}
Operation compleated successfully
